In [5]:
import pandas as pd
import torch as th
import os
import re
import yfinance as yf
import numpy as np
import warnings
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
warnings.filterwarnings('ignore')
            
import sys; sys.path.insert(0, '..')
from model.models import run_ensemble_strategy, TrainerConfig, Trainer
import data.preprocessing as pp
from utils.utils_analyze import get_price, check_index_dim
from utils.indicators import indicator_list, indicators_stock_stats
from env.BaseEnv import EnvConfig
from env.EnvStock_val import StockEnvValidation
from env.EnvStock_train import StockEnvTrain
from env.EnvStock_trade import StockEnvTrade
from policy.Policies import MlActorCriticPolicy
from config.config import indexes, ticker_list, category_dict, ticker_list_with_dict, tech_tickers
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Do not Run these two cell if you do not want to preprocess data from the begining

In [8]:
START_DATE = '2011-01-01'
SEED = 59

###  Run these two cell if you do not want to preprocess data from the begining

In [11]:
ppohparamlist = [{
    'ent_coef':0.005,
    'learning_rate':0.0005,

    },{
    'ent_coef':0.01,
    'learning_rate':0.00003,
    'vf_coef':0.005,
    'lam':0.90,
    'gamma': 0.99,#discountfactor
    'max_grad_norm':0.5,
    'cliprange':0.2,
    },{'gamma': 0.999,
 'ent_coef': 1.005,
 'cliprange': 0.4,
 'lam': 0.92,
 'max_grad_norm': 0.9,
 'vf_coef': 0.257}

]
n_actions = 30
ddpg = {
    'action_noise' : NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))
}
SAC_PARAMS = {"batch_size": 128,
              "buffer_size": 100000,"learning_rate": 0.0001,"learning_starts": 100,"ent_coef": "auto_0.1","seed":SEED,
              "device":"cuda"}
DATASET_VERSION = 'pretrainedTrial6'
MODEL_NAME = 'jim_sac-v14'
POPULATION = 1
TIME_INTERVAL = 1
START_TRADE = '2011-01-01'
policy_kwargs = dict(activation_fn=th.nn.ReLU,
                     net_arch=[dict(pi=[256, 128, 128], vf=[256, 128])])


In [12]:
tconfig = TrainerConfig(start_date=START_TRADE,
                        rebalance_window=63,
                        **{'hparams':SAC_PARAMS},
                        timesteps=50000,
                       #policy_kwargs = policy_kwargs,
                       index_list = indexes)

In [13]:
trainer = Trainer('SAC', 'MlpPolicy', 
                  StockEnvTrain, StockEnvValidation, StockEnvTrade, 
                  dataset_version=DATASET_VERSION, population=POPULATION,
                     config = tconfig, model_name= MODEL_NAME, debug=False)

Total number of stocks:   469
GPU available


In [14]:
NORNAMLIZE = True

In [16]:
trainer.train(dataset='datasets/downew30.csv', timesteps=35000, load=False, model_to_load='model_to_load', normalize =NORNAMLIZE)

======Model training from:  2011-01-01 to  2016-01-04T00:00:00.000000000
======Recurrent PPO Training for a population of 1========
Training time  jim_sac-v14 :  12.940760735670725  minutes
Best params,  {'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1', 'seed': 59, 'device': 'cuda'}
======Recurrent PPO Validation from:  2016-01-04T00:00:00.000000000 to  2016-04-05T00:00:00.000000000
-----------------
Total Reward:  -12.473183935508132
Total Trades:  1379
Mean Reward: -2.958457338014341
STD reward: 0.4415315722460013
-----------------
Total reward at validation for Reccurent PPO -12.473183935508132
Sharpe Ratio:  0
======Trading from:  2016-04-05T00:00:00.000000000 to  2016-07-05T00:00:00.000000000 Model is :  Rec_PPO
previous_total_asset:1000000
end_total_asset:1009391.3913207899
total_asset_change:9391.391320789931
Total cash is: 4.767669285367447$ and total holdings in stocks are 1009386.6236515045$
Buy & Hold strategy

date,▁
end_total_asset,▁
trade_reward,▁
date,1467676800000000000
end_total_asset,1009391.39132
trade_reward,0.0


======Model training from:  2011-01-01 to  2016-04-05T00:00:00.000000000
======Recurrent PPO Training for a population of 1========
Training time  jim_sac-v14 :  13.124311820665996  minutes
Best params,  {'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1', 'seed': 59, 'device': 'cuda'}
======Recurrent PPO Validation from:  2016-04-05T00:00:00.000000000 to  2016-07-05T00:00:00.000000000
-----------------
Total Reward:  -11.341304613277316
Total Trades:  1556
Mean Reward: -3.1112434394394426
STD reward: 0.4354122777882157
-----------------
Total reward at validation for Reccurent PPO -11.341304613277316
Sharpe Ratio:  0
======Trading from:  2016-07-05T00:00:00.000000000 to  2016-10-03T00:00:00.000000000 Model is :  Rec_PPO


2021-11-19 14:36:13.287 
  command:

    streamlit run C:\Users\kaanb\Anaconda3\envs\pytorchgpu\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


Saving to  results/account_value_trade_main_jim_sac-v14.csv
previous_total_asset:1009391.3913207899
end_total_asset:1057788.1616876626
total_asset_change:48396.77036687266
Total cash is: 23.94736461887447$ and total holdings in stocks are 1057764.2143230438$
Buy & Hold strategy with previous total asset:  1047752.845848743
Total Cost:  621.4344423160554
Sum of rewards  -621.434442316182
Total trades:  115
Total days in turbulance:  0
Sharpe:  0.1870446271123777
Total reward for the the window is -0.28470374416792765


...

======Model training from:  2011-01-01 to  2021-01-05T00:00:00.000000000
======Recurrent PPO Training for a population of 1========
Training time  jim_sac-v14 :  13.94423368771871  minutes
Best params,  {'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1', 'seed': 59, 'device': 'cuda'}
======Recurrent PPO Validation from:  2021-01-05T00:00:00.000000000 to  2021-04-07T00:00:00.000000000
-----------------
Total Reward:  -17.74656542018056
Total Trades:  486
Mean Reward: -3.732245735637844
STD reward: 0.06469106019261235
-----------------
Total reward at validation for Reccurent PPO -17.74656542018056
Sharpe Ratio:  0
======Trading from:  2021-04-07T00:00:00.000000000 to  2021-07-07T00:00:00.000000000 Model is :  Rec_PPO
Saving to  results/account_value_trade_main_jim_sac-v14.csv
previous_total_asset:1620676.437711485
end_total_asset:1761087.1564481603
total_asset_change:140410.71873667534
Total cash is: 11.35584620109038$ and 

date,▁
end_total_asset,▁
trade_reward,▁
date,1625616000000000000
end_total_asset,1761087.15645
trade_reward,0.0


======Model training from:  2011-01-01 to  2021-04-07T00:00:00.000000000
======Recurrent PPO Training for a population of 1========
Training time  jim_sac-v14 :  13.202977645397187  minutes
Best params,  {'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1', 'seed': 59, 'device': 'cuda'}
======Recurrent PPO Validation from:  2021-04-07T00:00:00.000000000 to  2021-07-07T00:00:00.000000000
-----------------
Total Reward:  -13.467477635233081
Total Trades:  1317
Mean Reward: -4.468473145026474
STD reward: 0.40005697798287915
-----------------
Total reward at validation for Reccurent PPO -13.467477635233081
Sharpe Ratio:  0
======Trading from:  2021-07-07T00:00:00.000000000 to  2021-10-05T00:00:00.000000000 Model is :  Rec_PPO
Saving to  results/account_value_trade_main_jim_sac-v14.csv
previous_total_asset:1761087.1564481603
end_total_asset:1715497.8757945003
total_asset_change:-45589.28065365995
Total cash is: 41.169148534826206

date,▁
end_total_asset,▁
trade_reward,▁
date,1633392000000000000
end_total_asset,1715497.87579
trade_reward,-0.00293


Ensemble Strategy took:  318.2538850824038  minutes


In [20]:

pretrain_set = pd.read_csv('datasets/pretrain_set3.csv')
pretrain_set['turbulence'] = 0

In [ ]:
pretrain = trainer.pretrain(sn, '2011-05-01', START_TRADE, NORNAMLIZE)

In [67]:
model_to_load = pretrain.split('/')[1]

In [16]:
trainer.cluster(False, period=365, number_of_clusters=5, stocks_per_cluster=5)

Loading processed path
Stocks trading this period is  ['AIG' 'AIV' 'AXP' 'BAC' 'BAX' 'BK' 'BMY' 'COG' 'CSCO' 'CSX' 'HAL' 'JNJ'
 'KO' 'LOW' 'MA' 'MSFT' 'NEM' 'NFLX' 'NVDA' 'PEP' 'PG' 'SCHW' 'UNH' 'USB'
 'V']
======Model training from:  2011-01-01 to  2012-05-07T00:00:00.000000000
======Recurrent PPO Training for a population of 1========
Training time  alameda-v9.1 :  6.870966609319051  minutes
Best params,  {'ent_coef': 0.005}
======Recurrent PPO Validation from:  2012-05-07T00:00:00.000000000 to  2012-08-06T00:00:00.000000000
-----------------
Total Reward:  430.2842980623245
Total Trades:  876
Mean Reward: 430.2842980623245
STD reward: 0.0
-----------------
Total reward at validation for Reccurent PPO 430.2842980623245
Sharpe Ratio:  0
======Trading from:  2012-08-06T00:00:00.000000000 to  2012-11-06T00:00:00.000000000 Model is :  Rec_PPO
Saving to  results/account_value_trade_main_alameda-v9.1.csv
previous_total_asset:1000000
end_total_asset:1011028.6612429218
total_asset_change:110

date,▁
end_total_asset,▁
trade_reward,▁
date,1352160000000000000
end_total_asset,1011028.66124
trade_reward,-11.36495


Stocks trading this period is  ['AMD' 'CAT' 'COG' 'CSCO' 'EW' 'EXC' 'GILD' 'GM' 'HIG' 'HPQ' 'INTC' 'KR'
 'LLY' 'MDLZ' 'MNST' 'MRK' 'MRO' 'NEE' 'NEM' 'NFLX' 'T' 'TJX' 'TWX' 'WMT'
 'WU']
======Model training from:  2011-01-01 to  2012-08-06T00:00:00.000000000
======Recurrent PPO Training for a population of 1========
Training time  alameda-v9.1 :  6.742207300662995  minutes
Best params,  {'ent_coef': 0.005}
======Recurrent PPO Validation from:  2012-08-06T00:00:00.000000000 to  2012-11-06T00:00:00.000000000
-----------------
Total Reward:  -441.5967130661011
Total Trades:  829
Mean Reward: -441.5967130661011
STD reward: 0.0
-----------------
Total reward at validation for Reccurent PPO -441.5967130661011
Sharpe Ratio:  0
======Trading from:  2012-11-06T00:00:00.000000000 to  2013-02-07T00:00:00.000000000 Model is :  Rec_PPO
Previously traded stocks are ['AMD' 'CAT' 'COG' 'CSCO' 'EW' 'EXC' 'GILD' 'GM' 'HIG' 'HPQ' 'INTC' 'KR'
 'LLY' 'MDLZ' 'MNST' 'MRK' 'MRO' 'NEE' 'NEM' 'NFLX' 'T' 'TJX' 'T

date,▁
end_total_asset,▁
trade_reward,▁
date,1360195200000000000
end_total_asset,907601.75694
trade_reward,-404.63809


Stocks trading this period is  ['AMD' 'CAT' 'COG' 'CSCO' 'EW' 'EXC' 'GILD' 'GM' 'HIG' 'HPQ' 'INTC' 'KR'
 'LLY' 'MDLZ' 'MNST' 'MRK' 'MRO' 'NEE' 'NEM' 'NFLX' 'T' 'TJX' 'TWX' 'WMT'
 'WU']
======Model training from:  2011-01-01 to  2012-11-06T00:00:00.000000000
======Recurrent PPO Training for a population of 1========
Training time  alameda-v9.1 :  5.894472555319468  minutes
Best params,  {'ent_coef': 0.005}
======Recurrent PPO Validation from:  2012-11-06T00:00:00.000000000 to  2013-02-07T00:00:00.000000000
Closing positions
Closing positions
Closing positions
Closing positions
Closing positions
Closing positions
Closing positions
Closing positions
Closing positions
Closing positions
Closing positions
-----------------
Total Reward:  -11800.243874296546
Total Trades:  477
Mean Reward: -11800.243874296546
STD reward: 0.0
-----------------
Total reward at validation for Reccurent PPO -11800.243874296546
Sharpe Ratio:  0
======Trading from:  2013-02-07T00:00:00.000000000 to  2013-05-09T00:0

...

Stocks trading this period is  ['AMD' 'COG' 'CVS' 'EQT' 'FE' 'FTI' 'GILD' 'IVZ' 'JNJ' 'KHC' 'KR' 'LUV'
 'MAC' 'MDLZ' 'NEM' 'NFLX' 'NVDA' 'OXY' 'QCOM' 'RF' 'SCHW' 'TSLA' 'V'
 'WFC' 'WMB']
======Model training from:  2011-01-01 to  2020-11-16T00:00:00.000000000
======Recurrent PPO Training for a population of 1========
Training time  alameda-v9.1 :  5.386644228299459  minutes
Best params,  {'ent_coef': 0.005}
======Recurrent PPO Validation from:  2020-11-16T00:00:00.000000000 to  2021-02-19T00:00:00.000000000
Closing positions
Closing positions
Closing positions
Closing positions
Closing positions
Closing positions
Closing positions
Closing positions
Closing positions
Closing positions
Closing positions
-----------------
Total Reward:  -7700.026377916336
Total Trades:  960
Mean Reward: -7560.539224982262
STD reward: 0.0
-----------------
Total reward at validation for Reccurent PPO -7700.026377916336
Sharpe Ratio:  0
======Trading from:  2021-02-19T00:00:00.000000000 to  2021-05-20T00:00

date,▁
end_total_asset,▁
trade_reward,▁
date,1621468800000000000
end_total_asset,4114075.99161
trade_reward,-69.45296


Stocks trading this period is  ['AMD' 'COG' 'CVS' 'EQT' 'FE' 'FTI' 'GILD' 'IVZ' 'JNJ' 'KHC' 'KR' 'LUV'
 'MAC' 'MDLZ' 'NEM' 'NFLX' 'NVDA' 'OXY' 'QCOM' 'RF' 'SCHW' 'TSLA' 'V'
 'WFC' 'WMB']
======Model training from:  2011-01-01 to  2021-02-19T00:00:00.000000000
======Recurrent PPO Training for a population of 1========
Training time  alameda-v9.1 :  5.340818456808726  minutes
Best params,  {'ent_coef': 0.005}
======Recurrent PPO Validation from:  2021-02-19T00:00:00.000000000 to  2021-05-20T00:00:00.000000000
-----------------
Total Reward:  -925.5321799591184
Total Trades:  656
Mean Reward: -925.5321799591184
STD reward: 0.0
-----------------
Total reward at validation for Reccurent PPO -925.5321799591184
Sharpe Ratio:  0
======Trading from:  2021-05-20T00:00:00.000000000 to  2021-08-19T00:00:00.000000000 Model is :  Rec_PPO
Saving to  results/account_value_trade_main_alameda-v9.1.csv
previous_total_asset:4114075.991613965
end_total_asset:4308427.727949969
total_asset_change:194351.7363

date,▁
end_total_asset,▁
trade_reward,▁
date,1629331200000000000
end_total_asset,4308427.72795
trade_reward,34.30508


Ensemble Strategy took:  190.28480036258696  minutes


In [60]:
m = pd.read_csv('./datasets/.csv')

In [102]:
from datetime import timedelta
pd.to_datetime('2011-12-30 00:00:00') + timedelta(days=4)

Timestamp('2012-01-03 00:00:00')

In [22]:
len(['KO', 'T', 'MRK', 'GILD', 'VZ', 'MSFT', 'CSCO', 'LOW', 'EBAY', 'XOM', 'NFLX', 'HAL', 'CSX', 'CMCSA', 'BAC', 'NVDA', 'HPQ', 'MS', 'GM', 'V', 'AAPL', 'SBUX', 'INTC', 'ABT'])

24

In [61]:
m

,Unnamed: 0,level_0,index,Date,Open,High,Low,Close,adjcp,volume,...,macd_30,signal_30,macd_60,signal_60,KAMA,MFI,category,turbulence,month,day
0,0,2100,84,2012-01-03,23.770000,24.500000,23.730000,24.070000,20.063438,4221700.0,...,-0.030042,-0.020859,-0.106061,-0.053554,22.960972,54.348434,3.0,0.0,1,3
1,1,2101,417,2012-01-03,3.116891,3.146220,3.072897,3.094227,1.725873,8963773.0,...,-0.000505,0.004060,0.023888,0.044236,2.987355,63.698944,8.0,0.0,1,3
2,2,2102,750,2012-01-03,48.299999,48.959999,48.139999,48.389999,41.869740,6955400.0,...,0.025726,0.033231,0.125205,0.138658,47.400181,51.218262,3.0,0.0,1,3
3,3,2103,1083,2012-01-03,5.750000,5.890000,5.740000,5.800000,5.042953,246293200.0,...,0.031555,0.040695,0.232891,0.262707,5.404477,57.136018,3.0,0.0,1,3
4,4,2104,1416,2012-01-03,27.229767,27.577404,27.229767,27.452471,23.178144,6193308.0,...,0.093604,0.111523,0.645550,0.694796,26.824866,36.333783,4.0,0.0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61745,61745,7420,6235,2021-11-02,81.879997,82.519997,81.379997,82.315697,82.315697,739170.0,...,-0.499344,-0.469403,-3.067288,-2.777909,80.895006,80.488804,2.0,0.0,11,2
61746,61746,7421,6532,2021-11-02,1159.349976,1208.579956,1146.000000,1175.875000,1175.875000,25335354.0,...,-17.907999,-12.730322,-99.439317,-73.040845,1134.862782,90.645390,1.0,0.0,11,2
61747,61747,7422,6829,2021-11-02,213.160004,213.149994,207.910004,209.649994,209.649994,5609056.0,...,0.599654,0.321424,3.467486,2.238154,220.253621,34.644438,5.0,0.0,11,2
61748,61748,7423,7126,2021-11-02,51.540001,51.709999,51.250000,51.669998,51.669998,5524292.0,...,-0.229830,-0.178914,-1.350323,-1.077521,50.236655,68.621784,2.0,0.0,11,2


In [4]:
#df1 = pp.add_turbulance(sn.rename(columns={'adjcp':'Adj Close'}))
#df_non_zero = df1.rename(columns={'Adj Close':'adjcp','Volume':'volume'}).replace([np.inf, -np.inf], 0)
df_non_zero['Date'] = pd.to_datetime(df_non_zero['Date'])
df_non_zero = df_non_zero.sort_values(['Date','ticker'])
#index_df['Date'] = pd.to_datetime(index_df['Date'])

unique_trade_date = df_non_zero[(df_non_zero.Date > pd.to_datetime('2005-02-02'))&(df_non_zero.Date <= pd.to_datetime('2021-06-20'))].Date.unique()
df_non_zero = df_non_zero[df_non_zero.Date >= START_DATE] 
#index_df = index_df[index_df.Date >= START_DATE].rename(columns={'index_ticker':'ticker'})
end_test = '2021-06-28'
index_dim = len(indexes)
# rebalance_window is the number of months to retrain the model
# validation_window is the numebr of months to validation the model and select for trading
rebalance_window = 63
validation_window = 63

NameError: name 'df_non_zero' is not defined